In [145]:
# Imports
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.models import MobileNet_V2_Weights
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, Subset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

import os

from sklearn.model_selection import StratifiedShuffleSplit

import numpy as np

In [146]:
# Variables
BATCH_SIZE = 32
RANDOM_STATE = 42
TEST_SIZE = 0.2

TRAIN_DIR = '../prep/preprocessed_images/train'
TEST_DIR = '../prep/preprocessed_images/test'

EPOCHS = 10

STEP_SIZE = 5  
GAMMA = 0.1
lr = 0.001


In [147]:
# Set device: use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [148]:
# Load pre-trained MobileNetV2
model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)

In [149]:
# Learning rate scheduler
optimizer = Adam(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [150]:
# Freeze the convolutional layers (feature extraction part)
for param in model.parameters():
    param.requires_grad = False

In [151]:
# Modify the classifier part to suit your task (3 classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 3)  # 3 classes: rock, paper, scissors

In [152]:
# Move the model to the GPU (if available)
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [153]:
# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.classifier.parameters(), lr=0.001)

In [154]:
# Define a transformation for the images
transform = transforms.Compose([
    transforms.ToTensor()
])

In [155]:
# Load the training dataset
full_training_dataset = ImageFolder(root=TRAIN_DIR, transform=transform)  # No need for separate folders

In [156]:
# Extract only the labels from the dataset
targets = [full_training_dataset.targets[i] for i in range(len(full_training_dataset))]

# Define the stratified split
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Obtain train and validation indices
for train_idx, val_idx in stratified_split.split(np.zeros(len(targets)), targets):
    train_set = Subset(full_training_dataset, train_idx)
    val_set = Subset(full_training_dataset, val_idx)


In [157]:
# Load all the data
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(ImageFolder(root=TEST_DIR, transform=transform), batch_size=BATCH_SIZE, shuffle=False)

print('Training set size:', len(train_set))
print('Validation set size:', len(val_set))
print('Test set size:', len(test_loader.dataset))

Training set size: 414
Validation set size: 104
Test set size: 133


In [158]:
# Training loop
for epoch in range(EPOCHS):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, label in train_loader:
        images, labels = images.to(device), label.to(device)

        optimizer.zero_grad()  # Zero the gradients
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights

        running_loss += loss.item() * images.size(0)
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Update the scheduler after the optimizer step
    scheduler.step()

    train_loss = running_loss / len(train_set)
    train_accuracy = 100. * correct / total
    print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            output = model(images)
            loss = criterion(output, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = output.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_set)
    val_accuracy = 100. * correct / total
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')


c:\Users\Timoy\anaconda3\envs\minor\Lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/10, Loss: 1.0465, Accuracy: 43.24%
Validation Loss: 0.9867, Accuracy: 50.00%
Epoch 2/10, Loss: 0.9038, Accuracy: 72.46%
Validation Loss: 0.8726, Accuracy: 74.04%
Epoch 3/10, Loss: 0.8042, Accuracy: 77.05%
Validation Loss: 0.7953, Accuracy: 80.77%
Epoch 4/10, Loss: 0.7307, Accuracy: 82.61%
Validation Loss: 0.7277, Accuracy: 82.69%
Epoch 5/10, Loss: 0.6445, Accuracy: 86.23%
Validation Loss: 0.6724, Accuracy: 81.73%
Epoch 6/10, Loss: 0.6056, Accuracy: 88.16%
Validation Loss: 0.6298, Accuracy: 82.69%
Epoch 7/10, Loss: 0.5536, Accuracy: 86.96%
Validation Loss: 0.5867, Accuracy: 86.54%
Epoch 8/10, Loss: 0.5133, Accuracy: 90.10%
Validation Loss: 0.5589, Accuracy: 85.58%
Epoch 9/10, Loss: 0.4830, Accuracy: 90.82%
Validation Loss: 0.5284, Accuracy: 86.54%
Epoch 10/10, Loss: 0.4671, Accuracy: 90.58%
Validation Loss: 0.4985, Accuracy: 86.54%


In [159]:
# Test loop
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_loss /= len(test_loader.dataset)
test_accuracy = 100. * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.3918, Test Accuracy: 96.24%


In [160]:
# Save the model
torch.save(model.state_dict(), './model/model.pth')